# Rebalance XLM

In [3]:
import csv
import json
import time
import hmac
import random
import hashlib
import requests
import numpy as np
import pandas as pd
from datetime import datetime
from songline import Sendline
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
class BitkubRebalance():
    
    def __init__(self, API_HOST, API_KEY, API_SECRET, line_token, symbol=None):
        #API Bitkub
        self.API_HOST = API_HOST
        self.API_KEY = API_KEY
        self.API_SECRET = bytes(API_SECRET, encoding='utf-8')
        self.header = {
            'Accept': 'application/json',
            'Content-Type': 'application/json',
            'X-BTK-APIKEY': API_KEY,
        }
        self.symbol = symbol
        
        self.msg = Sendline(line_token) #Send line
        
    def json_encode(self, data):
        return json.dumps(data, separators=(',', ':'), sort_keys=True)

    def sign(self, data):
        j = self.json_encode(data)
        #print('Signing payload: ' + j)
        h = hmac.new(self.API_SECRET, msg=j.encode(), digestmod=hashlib.sha256)
        return h.hexdigest()

    def timeserver(self):
        response = requests.get(self.API_HOST + '/api/servertime')
        ts = int(response.text)
        return ts
        
    def allSymbol(self):
        response = requests.get(self.API_HOST + '/api/market/symbols')
        symbols = pd.DataFrame(response.json()['result'])
        return symbols
        
    def getPrice(self):
        pair = self.symbol
        response = requests.get(self.API_HOST + '/api/market/ticker')
        price = float(response.json()[pair]['last'])
        return price
    
    def createBuy(self, amount, rate, typ):
        data = {
            'sym': self.symbol, # THB_XLM
            'amt': amount, # THB amount you want to spend 
            'rat': rate, # Price
            'typ': typ, # limit Order
            'ts': self.timeserver(),
        }
        
        signature = self.sign(data)
        data['sig'] = signature
        r = requests.post(self.API_HOST + '/api/market/place-bid', headers=self.header, data=self.json_encode(data))
        r = r.json()
        
        if r['error'] == 0:
            print(f'Buy {typ}: {self.symbol} @{rate:.3f} amount: {amount:.3f} THB')
            #self.msg.sendtext(f'Buy {typ}: {self.symbol} @{rate:.3f} amount: {amount:.3f} THB')
            
        elif r['error'] == 18:
            print(f'Error 18: Insufficient balance.')
            #self.msg.sendtext('Error 18: Insufficient balance.')
        
        elif r['error'] == 15:
            print('Error 15: Amount too low.')
            #self.msg.sendtext('Error 15: Amount too low.')
        
        else:
            print('Error: Please check the code.')
            #self.msg.sendtext('Error: Please check the code.')

    
    def createSell(self, amount, rate, typ):
        data = {
            'sym': self.symbol, # THB_XLM
            'amt': amount, # XLM unit you want to sell 
            'rat': rate, # Price
            'typ': typ, # limit Order
            'ts': self.timeserver(),
        }
        
        signature = self.sign(data)
        data['sig'] = signature
        r = requests.post(self.API_HOST + '/api/market/place-ask', headers=self.header, data=self.json_encode(data))
        r = r.json()
        
        if r['error'] == 0:
            print(f'Sell {typ}: {self.symbol} @{rate:.3f} amount: {amount:.3f} unit')
            #self.msg.sendtext(f'Sell {typ}: {self.symbol} @{rate:.3f} amount: {amount:.3f} unit')
            
        elif r['error'] == 18:
            print(f'Error 18: Insufficient balance.')
            #self.msg.sendtext('Error 18: Insufficient balance.')
        
        elif r['error'] == 15:
            print('Error 15: Amount too low.')
            #self.msg.sendtext('Error 15: Amount too low.')
        
        else:
            print('Error: Please check the code.')
            #self.msg.sendtext('Error: Please check the code.')
        
        
    def my_open_orders(self):
        data = {
            'sym': self.symbol,
            'ts' : self.timeserver(),
        }
        
        signature = self.sign(data)
        data['sig'] = signature
        r = requests.post(self.API_HOST + '/api/market/my-open-orders', headers=self.header, data=self.json_encode(data))
        r = r.json()['result']
        df_r =  pd.DataFrame(r, columns=['id', 'side', 'type', 'rate', 'fee', 'amount', 'receive'])
        return df_r 
        
    def cancelOrder(self, order_id, side):
        data = {
            'sym': self.symbol,
            'id': order_id,
            'sd': side,
            'ts' : self.timeserver(),
        }
        
        signature = self.sign(data)
        data['sig'] = signature
        r = requests.post(self.API_HOST + '/api/market/cancel-order', headers=self.header, data=self.json_encode(data))
        r = r.json()
        
        if r['error'] == 21:
            print(f'Order ID: {order_id} Invalid order for cancellation.')
            #self.msg.sendtext(f'Order ID: {order_id} Invalid order for cancellation.')
            
        elif r['error'] == 0:
            print(f'Order ID: {order_id} has been cancelled.')
            #self.msg.sendtext(f'Order ID: {order_id} has been cancelled.')
        
        else:
            print('Cancel error please check the code.')
            #self.msg.sendtext('Cancel error please check the code.')
            
            
    def wallet(self): # Get user available balances
        data = {
            'ts': self.timeserver(),
        }
        
        signature = self.sign(data)
        data['sig'] = signature
        r = requests.post(self.API_HOST + '/api/market/wallet', headers=self.header, data=self.json_encode(data))
        r = r.json()['result']
        return r
    
    def balances(self): 
        # Includes both available and reserved balances
        data = {
            'ts': self.timeserver(),
        }
        
        signature = self.sign(data)
        data['sig'] = signature
        r = requests.post(self.API_HOST + '/api/market/balances', headers=self.header, data=self.json_encode(data))
        r = r.json()['result']
        df_total = pd.DataFrame.from_dict(r) # .sum(axis=0)
        
        return df_total
    
    def order_history(self):
        data = {
            'sym': self.symbol,
            'ts': self.timeserver(),
        }
        
        signature = self.sign(data)
        data['sig'] = signature
        r = requests.post(self.API_HOST + '/api/market/my-order-history', headers=self.header, data=self.json_encode(data))
        r = r.json()
        return r
    
    def cancelAllOrder(self):
        order_id = self.my_open_orders()['id']
        side = self.my_open_orders()['side']
        for i,j in zip(order_id,side):
            self.cancelOrder(i, j)
    
    def order_info(self, order_id, side):
        # For check status
        data = {
            'sym': self.symbol,
            'id': order_id,
            'sd': side,
            'ts': self.timeserver(),
        }
        
        signature = self.sign(data)
        data['sig'] = signature
        r = requests.post(self.API_HOST + '/api/market/order-info', headers=self.header, data=self.json_encode(data))
        r = r.json()
        return r
    
    def rebalance(self, min_diff, fixed, rp):
        # rebalance every diff% value asset
        # fixed=asset value fixed
        # rp = risk parameter (first entry buy)
        
        quote = self.symbol.split('_')[0]
        based = self.symbol.split('_')[1]
        
        price = self.getPrice()
        cash = self.balances()[quote].sum(axis=0)
        asset = self.balances()[based].sum(axis=0)
        
        asset_val = price*asset
        
        diff = asset_val - fixed
        mdf = min_diff*fixed # min diff value 2% of fixed asset value
        
        print(f'Asset:{asset:.4f} x Price:{price:.2f} = Value:{asset_val:.4f} ==> Diff:{diff:.4f}')
        
        # Sell
        if diff>mdf:
            if self.my_open_orders().empty: 
                pass
            else:
                self.cancelAllOrder()
            
            print(f'sell:{price}, amount:{diff}')
            unit = diff/price
            #self.createSell(unit, price+price*0.001, 'limit')
            
        # Buy
        elif diff<-mdf and price<=rp:
            if self.my_open_orders().empty: 
                pass
            else:
                self.cancelAllOrder()
            print(f'buy:{price}, amount:{abs(diff)}')
            #self.createBuy(abs(diff), price-price*0.001, 'limit')
        
        #Wait
        else:
            print('Wait !!!')
            #self.msg.sendtext('Wait!!!')
            
        
    def botLineStart(self):
        self.msg.sendtext('BOT START WORKING')
        self.msg.sendimage('https://i0.wp.com/ozarkwebdesign.com/wp-content/uploads/2019/10/business-memes-001.jpg?w=730')
        

In [8]:
BotBK = BitkubRebalance(API_HOST='https://api.bitkub.com',
            API_KEY='15532eb7e3a3934d19ce85a121c38e85', 
            API_SECRET='ec3a0a80a1e1c69ada837dc89aa4af85',
            line_token='8qiLtlAzfUisx9YmZ57WVEoLTaKiW3YFziSlx2WQKVg',
            symbol='THB_XLM')

In [10]:
tb = 60 # Rebalance every 600s = 10min
min_diff = 0.02
fixed = 1000
rp = 12
 
#BotBK.botLineStart()

while True:
 
    BotBK.rebalance(min_diff, fixed, rp)
    print(f'Sleep {tb} sec')
    print('-'*40)
 
    time.sleep(tb)

Asset:0.0000 x Price:10.90 = Value:0.0000 ==> Diff:-1000.0000
buy:10.9, amount:1000.0
Sleep 60 sec
----------------------------------------
Asset:0.0000 x Price:10.89 = Value:0.0000 ==> Diff:-1000.0000
buy:10.89, amount:1000.0
Sleep 60 sec
----------------------------------------


KeyboardInterrupt: 

In [6]:
BotBK.createSell(48, 13, 'limit')

Sell limit: THB_XLM @13 amount: 48 THB
